In [15]:
from owlready2 import *
import pandas as pd

In [2]:
onto = get_ontology("BreastCancerTreatments.owl").load()

In [3]:
print(onto)

get_ontology("http://www.semanticweb.org/kaixuankhoo/ontologies/2022/6/BreastCancerOntology#")


In [4]:
sync_reasoner_pellet()

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\antlr-3.2.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;c:\Users\Kaixuan Khoo\AppData\Local\Programs\Python\Python38\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;c:\Users\Kaixua

In [5]:
print(Thing)

owl.Thing


In [6]:
# print(onto)
print(onto.Patient1.PatientID)

[10262332]


## Playing around with Querying

In [7]:
## searching using IRI name
onto.search(iri = "*REF")

[BreastCancerTreatments.CyclophosphamideREF, BreastCancerTreatments.CapecitabineREF, BreastCancerTreatments.CarboplatinREF, BreastCancerTreatments.DocetaxelREF, BreastCancerTreatments.EpirubicinREF, BreastCancerTreatments.Fluorouracil5REF, BreastCancerTreatments.InfusionREF, BreastCancerTreatments.IntramuscularREF, BreastCancerTreatments.IntravenousREF, BreastCancerTreatments.PaclitaxelREF, BreastCancerTreatments.PertuzumabREF, BreastCancerTreatments.SubcutaneousREF, BreastCancerTreatments.TrastuzumabREF]

In [8]:
## searching relationships using object properties
onto.search(has_tumour = "*")

[BreastCancerTreatments.Patient1]

In [9]:
## searching subclass of something
onto.search(subclass_of = onto.Drug_reference)

[BreastCancerTreatments.Drug_reference, BreastCancerTreatments.AlkylatingAgent, BreastCancerTreatments.Anthracycline, BreastCancerTreatments.AntiMetabolite, BreastCancerTreatments.Taxane, BreastCancerTreatments.HER2Targeted, BreastCancerTreatments.PlatinumBased, BreastCancerTreatments.NitrogenMustardBased, BreastCancerTreatments.Epirubicin, BreastCancerTreatments.Capecitabine, BreastCancerTreatments.FU, BreastCancerTreatments.Docetaxel, BreastCancerTreatments.Paclitaxel, BreastCancerTreatments.Pertuzumab, BreastCancerTreatments.Trastuzumab, BreastCancerTreatments.Carboplatin, BreastCancerTreatments.Cyclophosphamide]

## Tests for creating individuals 
These individuals will then go into the ABox for reasoning

In [10]:
my_drug = onto.Drug("my_drug", Dose = [100], has_drug_reference = [onto.CyclophosphamideREF])
my_drug.Dose = [200] ## reassigning datapoint
my_drug.has_drug_reference = [onto.Fluorouracil5REF] ## reassigning relationship property

In [11]:
print(my_drug.has_drug_reference)

[BreastCancerTreatments.Fluorouracil5REF]


In [12]:
## For printing the properties of a drug
for property in my_drug.get_properties():
    for value in property[my_drug]:
        print(".%s == %s" % (property.python_name, value))

.Dose == 200
.has_drug_reference == BreastCancerTreatments.Fluorouracil5REF


In [13]:
for i in onto.Drug_reference.instances():
    print(i)

BreastCancerTreatments.CapecitabineREF
BreastCancerTreatments.CarboplatinREF
BreastCancerTreatments.CyclophosphamideREF
BreastCancerTreatments.DocetaxelREF
BreastCancerTreatments.EpirubicinREF
BreastCancerTreatments.Fluorouracil5REF
BreastCancerTreatments.PaclitaxelREF
BreastCancerTreatments.PertuzumabREF
BreastCancerTreatments.TrastuzumabREF


In [14]:
## Printing domain and range
print(onto.treats.domain)
print(onto.treats.range)

[BreastCancerTreatments.Regimen]
[BreastCancerTreatments.Tumour]


## Trying to import data and mapping into the ontology
Using a dummy dataset imported into pandas

In [16]:
df = pd.read_csv('data/dummy_breast.csv')

In [57]:
destroy_entity(onto.trial1)

In [59]:
## Trial creating a new individual
test_row = df.head(1)
type(test_row['MERGED_TUMOUR_ID'][0].item())
trial_patient = onto.Patient("trialP", PatientID = [test_row['MERGED_PATIENT_ID_x'][0].item()])
print(trial_patient.PatientID)
trial_tumour = onto.Tumour("trialT", TumourID = [test_row['MERGED_TUMOUR_ID'][0].item()])
print(trial_tumour.TumourID)
trial_regimen = onto.Regimen("trialR", RegimenID = [test_row['MERGED_REGIMEN_ID'][0].item()])
print(trial_regimen.RegimenID)

[10231909]
[10000032]
[10000046]


In [27]:
## trying to import them in a for loop; this may be a slow way to do it...
#TODO: Add DOB, find a way to map text to the drug instance ... ???

for index, row in df.iterrows():
    print(row['MERGED_REGIMEN_ID'], row['MERGED_TUMOUR_ID'], row['MERGED_PATIENT_ID_x'], row['PRIMARY_DIAGNOSIS'], row['MAPPED_REGIMEN'])
    # Create a new patient instance
    # Create a new tumour instance
    # Create new drug instances in a for loop using names
    # Create a new regimen instance
    # Connect them all

## Print out the time taken to do all 20 of the dummy dataset
    

10000046 10000032 10231909 C509 Capecitabine
10000047 10000032 10231909 C509 Carboplatin + Etoposide IV 3 day
10000364 10000241 10271562 C509 Capecitabine
10000643 10000428 10288769 C509 Capecitabine + Oxaliplatin 21day
10000693 10000460 10290756 C509 Capecitabine
10000694 10000460 10290756 C509 CHOP R
10001010 10000662 10307019 C509 Capecitabine
10001054 10000696 10309417 C509 Capecitabine
10001055 10000696 10309417 C509 CHOP R
10001212 10000797 10343133 C509 Capecitabine
10001258 10000832 10346241 C509 FEC 100
10001301 10000855 10349302 C509 FEC 60 or 75
10001306 10000859 10349584 C509 FEC 100
10001326 10000873 10350855 C509 FEC
10001336 10000882 10351591 C509 FEC 100
10001337 10000882 10351591 C509 Carboplatin AUC 6 - 10
10001341 10000883 10351982 C509 FEC 100
10001357 10000895 10353081 C509 Docetaxel 75mg/m2 (21 day)
10001361 10005948 10353654 C509 FEC 100
10001365 10000900 10353965 C509 FEC 100
